In [ ]:
'''
LANGCHAIN_API_KEY = ...
OPENAI_API_KEY = ...
GOOGLE_API_KEY = ...
LANGCHAIN_PROJECT = "default"
'''

# pip install langchain_community tiktoken langchainhub chromadb langchain langchainhub

In [117]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
import os

In [109]:
os.environ['OPENAI_API_KEY'] = '...'
os.environ['GOOGLE_API_KEY'] = '...'
os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ['HUGGINGFACEHUB_API_TOKEN'] = '...'

In [38]:
llm = GoogleGenerativeAI(
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    model="models/gemini-pro",
    temperature=0.9,
    # max_output_tokens=500
)
llm.invoke('HEY')

'Hello there! How can I assist you today?'

In [39]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-pro",
)
llm.invoke("hey")

AIMessage(content='Hi there! How can I help you today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-5b7574f3-5b97-4045-854c-a9523b19039e-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12})

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)


# Part 1: Overview

In [30]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

D:\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Python_(programming_language)")
docs = loader.load()
docs

[Document(page_content='\n\n\n\nPython (programming language) - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory\n\n\n\n\n\n\n\n\n2\nDesign philosophy and features\n\n\n\n\n\n\n\n\n3\nSyntax and semantics\n\n\n\n\nTog

In [41]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Python_(programming_language)")
docs = loader.load()

## SPLIT
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


## Embed
# Embedembeddings_g = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# vectorstore = Chroma.from_documents(documents = splits, embedding=GoogleGenerativeAIEmbeddings(google_api_key= os.getenv("GOOGLE_API_KEY"), model="models/embedding-001"))
vectorstore = Chroma.from_documents(documents = splits, embedding=hf)

retriever = vectorstore.as_retriever()
## retriever is set to the result of calling the as_retriever() 
# method on the vectorstore object. This means that retriever 
# can be used to find documents in the vector store that are 
# semantically similar to a given query. 


## PROMPT
prompt = hub.pull('rlm/rag-prompt')


## LLM
llm = ChatGoogleGenerativeAI(
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    model="models/gemini-pro",
    temperature=0.9
)


## Post-processing
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)


## CHAIN
rag_chain = (
    {'context':retriever| format_docs, 'question': RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)


## QUESTION
rag_chain.invoke("when was python invented")

'Python was invented in the late 1980s by Guido van Rossum. The implementation of the language began in December 1989.'

# Part 2: Indexing

In [42]:
question = 'What kind of pets do I like?'
document = 'My favourite pet is a cat'

In [48]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4o")

In [52]:
import tiktoken

def num_tokens_from_string(string: str,encoding_name: str)->int:
    enc = tiktoken.get_encoding(encoding_name)
    encoding = enc.encode(string)
    print(encoding)
    nums_tokens = len(encoding)

    return nums_tokens

num_tokens_from_string(question, 'cl100k_base')

[3923, 3169, 315, 26159, 656, 358, 1093, 30]


8

## text embeddings

In [55]:
from langchain_huggingface import HuggingFaceEmbeddings

embd = HuggingFaceEmbeddings()

query_result = embd.embed_query(question)
document_result = embd.embed_query(document)

len(query_result)

D:\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


768

In [107]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)

## `Cosine Similarity`:
 is recommended for OpenAI embeddings. 1 means identical

In [58]:
import numpy as np

def cosine_similarity(vec1,vec2):
    dot_product = np.dot(vec1,vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print('cosine_similarity:',similarity)

cosine_similarity: 0.5844920257124517


## Document Loader

SoupStrainer: Parsing only part of a document.

You just create a SoupStrainer and pass it in to the BeautifulSoup constructor as the parse_only argument.

class_ == html classes in the webpage

In [95]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ('post-content','post-title','post-header')
        )
    ),   
)


blog_docs = loader.load()

## Text Splitter

splitting based on tokens

In [96]:
## split

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300,
    chunk_overlap = 50
)

## Make splits

splits = text_splitter.split_documents(blog_docs)

## Vectorstore

In [115]:
## Index
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding = HuggingFaceEmbeddings())

retriever = vectorstore.as_retriever()


D:\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Part 3: Retrieval

In [116]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

retriever = vectorstore.as_retriever(search_kwargs={'k':1})
# k nearest neighbors

D:\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [72]:
# docs = retriever.get_relevant_documents('what is Task Decomposition?')
docs = retriever.invoke('what is Task Decomposition?')

In [73]:
len(docs)

3

# Part 4: Genaration

In [75]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = '''Answer the questions based only on the following context:
{context}
Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the questions based only on the following context:\n{context}\nQuestion: {question}\n'))])

In [118]:
# LLM

llm = ChatGoogleGenerativeAI(model='models/gemini-pro',temperature=1)
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)

In [119]:
# Cha

chain = prompt|llm

In [120]:
# Run

chain.invoke({'context' : docs, "question":"what is Task Decompostion"})

AIMessage(content='Task Decomposition refers to the process of breaking down user requests or tasks into multiple sub-tasks or components, each with its own attributes such as task type, ID, dependencies, and arguments. This process is carried out by the LLM system in the context of how HuggingGPT works.', response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 399, 'total_tokens': 459}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-235d7bf5-e1c3-41c1-93cf-c17c003b843e-0', usage_metadata={'input_tokens': 399, 'output_tokens': 60, 'total_tokens': 459})

In [121]:
from langchain import hub
prompt_hub_rag = hub.pull('rlm/rag-prompt')

In [122]:
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

### RAG chains

In [124]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

rag_chain.invoke('what is task decomposition')

'Task decomposition is the process of breaking down a task into smaller subtasks or components, as described in the context provided.'